<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Armazenamento e Gestão de Dados com Data Lake e Data Lakehouse</font>
## <font color='blue'>Projeto Final</font>
### <font color='blue'>Consolidação, Limpeza e Enriquecimento de Dados de Diferentes Formatos no Data Lakehouse</font>

## Tabela Silver de Clientes

In [ ]:
# Define o banco de dados que será usado
spark.sql('use database dsa_db_02_staging')

In [ ]:
# Cria a tabela Silver
spark.sql("""create table if not exists dsa_silver_clientes(
             customer_id integer,
             name string,
             email string,
             country string,
             customer_type string,
             registration_date date,
             age integer,
             gender string,
             total_purchases integer,
             last_updated_at timestamp
          )""")

In [ ]:
# Extrai a última data de atualização (a maior data)
last_updated_df = spark.sql('select max(last_updated_at) as last_completed from dsa_silver_clientes')

In [ ]:
# Extrai a data de atualização
last_updated_time = last_updated_df.collect()[0]['last_completed']

In [ ]:
# Se não houver data de atualização, considera a data de 1900-01-01
if last_updated_time is None:
    last_updated_time = '1900-01-01T00:00:00.000+00:00'

In [ ]:
# Vamos criar uma view temporária a fim de verificar que o registro recebido na tabela bronze foi ou não processado para a tabela silver
spark.sql(f"""
          create or replace temporary view clientes_incremental as
          select * from dsa_db_01_inicial.dsa_bronze_clientes as c where c.recebido_em  > '{last_updated_time}' """)

In [ ]:
# Select
spark.sql("select * from clientes_incremental limit 10").show()

In [ ]:
# Vamos criar uma view temporária somente para clientes que realizaram pelo menos uma compra
spark.sql("""
          create or replace temporary view vw_silver_clientes_incremental as
          select
          customer_id,
          case 
            when name is not null then initcap(trim(name))
            else 'Unknown'
          end as name,
          case 
            when email is not null then lower(trim(email))
            else null
          end as email,
          case 
            when country is not null then trim(country)
            else 'Unknown'
          end as country,
          case 
            when customer_type in ('Regular','Premium','VIP') then customer_type
            else 'Unknown'
          end as customer_type,
          registration_date,
          case
            when age between 18 and 100 then age
            else null
          end as age,
          case
            when gender in ('Male','Female','Other') then gender
            else null
          end as gender,
          case 
            when total_purchases > 0  then total_purchases
            else 0
          end as total_purchases,
          current_timestamp as last_updated_at
          from clientes_incremental 
          where customer_id is not null and total_purchases > 0 and email is not null""")

In [ ]:
# Visualiza os dados
display(spark.sql("select * from vw_silver_clientes_incremental limit 10"))

In [ ]:
# Agora fazemos o merge.
# Se o registro já havia sido processado, atualizamos as colunas.
# Caso contrário, inserimos o novo registro.
spark.sql("""
    MERGE INTO dsa_silver_clientes target
    USING vw_silver_clientes_incremental source
    ON target.customer_id = source.customer_id
    WHEN MATCHED THEN 
        UPDATE SET 
            target.name = source.name,
            target.email = source.email,
            target.country = source.country,
            target.customer_type = source.customer_type,
            target.registration_date = source.registration_date,
            target.age = source.age,
            target.gender = source.gender,
            target.total_purchases = source.total_purchases,
            target.last_updated_at = source.last_updated_at
    WHEN NOT MATCHED THEN 
        INSERT (customer_id, name, email, country, customer_type, registration_date, age, gender, total_purchases, last_updated_at)
        VALUES (source.customer_id, source.name, source.email, source.country, source.customer_type,  source.registration_date, source.age, source.gender, source.total_purchases, source.last_updated_at)
""")


In [ ]:
spark.sql("select count(*) from dsa_silver_clientes").show()

# Fim